## Verify if patients excluded through automatic exclusion in preprocessing and manual exclusion correspond

In [ ]:
import pandas as pd

In [ ]:
manual_exclusion_paths = ['/Users/jk1/temp/opsum_extration_output/ehr_extraction_errors/manual_verification/patients_with_missing_data.csv',
                          # '/Users/jk1/temp/opsum_extration_output/ehr_extraction_errors/manual_verification/patients_with_slightly_shifted_eds_start.csv',
                          # '/Users/jk1/temp/opsum_extration_output/ehr_extraction_errors/manual_verification/patients_with_stroke_after_7_days_admission.csv',
                          '/Users/jk1/temp/opsum_extration_output/ehr_extraction_errors/manual_verification/patients_with_wrong_eds_start_and_missing_data_removed.xlsx',
                          '/Users/jk1/temp/opsum_extration_output/ehr_extraction_errors/manual_verification/patients_with_wrong_eds_start.csv']

In [ ]:
preprocessing_excluded_patients_path = '/Users/jk1/temp/opsum_prepro_output/logs_15062022_191426/missing_cids_from_feature_database.tsv'

In [ ]:
manual_exclusion_dfs = []
for manual_exclusion_path in manual_exclusion_paths:
    if manual_exclusion_path.endswith('.csv'):
        manual_exclusion_dfs.append(pd.read_csv(manual_exclusion_path, dtype=str))
    elif manual_exclusion_path.endswith('.xlsx'):
        manual_exclusion_dfs.append(pd.read_excel(manual_exclusion_path, dtype=str))


In [ ]:
manual_exclusion_cids = []
for manual_exclusion_df in manual_exclusion_dfs:
    if 'patient_id_x' in manual_exclusion_df.columns:
        manual_exclusion_cids.append(manual_exclusion_df['patient_id_x'].astype(str) + '_' + manual_exclusion_df['EDS_last_4_digits_x'].astype(str))
    else:
        manual_exclusion_cids.append(manual_exclusion_df['patient_id'].astype(str) + '_' + manual_exclusion_df['EDS_last_4_digits'].astype(str))

In [ ]:
def flatten_list(l):
    return [item for sublist in l for item in sublist]

In [ ]:
manual_exclusion_cids = set(flatten_list(manual_exclusion_cids))

In [ ]:
preprocessing_excluded_patients_df = pd.read_csv(preprocessing_excluded_patients_path, sep='\t')

In [ ]:
preprocessing_excluded_patients_cids = set(preprocessing_excluded_patients_df['case_admission_id'].astype(str))

Find patients that are only excluded through preprocessing

In [ ]:
cids_only_excluded_through_preprocessing = preprocessing_excluded_patients_cids.difference(manual_exclusion_cids)

Find patients that are only excluded through manual exclusion

In [ ]:
cids_only_excluded_through_manual_exclusion = manual_exclusion_cids.difference(preprocessing_excluded_patients_cids)

In [ ]:
# for a given patient_id, find in which manual exclusion file the patient is
pid = '58752'
for idx, manual_exclusion_df in enumerate(manual_exclusion_dfs):
    if 'patient_id_x' in manual_exclusion_df.columns:
        if pid in manual_exclusion_df['patient_id_x'].astype(str).values:
            print(manual_exclusion_paths[idx])
    else:
        if pid in manual_exclusion_df['patient_id'].astype(str).values:
            print(manual_exclusion_paths[idx])